The purpose of this notebook is to attempt to create a CNN to classify dog breeds. 

### 1. Set Up

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!wget "https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip" -P "/content/drive/MyDrive/8_ModelRun/data"

--2021-01-02 02:14:49--  https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.117.32
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.117.32|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1132023110 (1.1G) [application/zip]
Saving to: ‘/content/drive/MyDrive/8_ModelRun/data/dogImages.zip’

dogImages.zip       100%[===================>]   1.05G  19.8MB/s    in 60s     

2021-01-02 02:15:49 (18.1 MB/s) - ‘/content/drive/MyDrive/8_ModelRun/data/dogImages.zip’ saved [1132023110/1132023110]



In [4]:
!unzip "/content/drive/MyDrive/8_ModelRun/data/dogImages.zip" -d "/content/drive/MyDrive/8_ModelRun/data/dogImages"

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/8_ModelRun/data/dogImages/dogImages/train/048.Chihuahua/Chihuahua_03417.jpg  
  inflating: /content/drive/MyDrive/8_ModelRun/data/dogImages/dogImages/train/048.Chihuahua/Chihuahua_03418.jpg  
  inflating: /content/drive/MyDrive/8_ModelRun/data/dogImages/dogImages/train/048.Chihuahua/Chihuahua_03421.jpg  
  inflating: /content/drive/MyDrive/8_ModelRun/data/dogImages/dogImages/train/048.Chihuahua/Chihuahua_03423.jpg  
  inflating: /content/drive/MyDrive/8_ModelRun/data/dogImages/dogImages/train/048.Chihuahua/Chihuahua_03424.jpg  
  inflating: /content/drive/MyDrive/8_ModelRun/data/dogImages/dogImages/train/048.Chihuahua/Chihuahua_03425.jpg  
  inflating: /content/drive/MyDrive/8_ModelRun/data/dogImages/dogImages/train/048.Chihuahua/Chihuahua_03426.jpg  
  inflating: /content/drive/MyDrive/8_ModelRun/data/dogImages/dogImages/train/048.Chihuahua/Chihuahua_03428.jpg  
  inflating: /content/drive/MyDrive/8

In [3]:
!wget "https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/lfw.zip" -P "/content/drive/MyDrive/8_ModelRun/data"

--2021-01-02 02:16:36--  https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/lfw.zip
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.112.136
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.112.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 196739509 (188M) [application/zip]
Saving to: ‘/content/drive/MyDrive/8_ModelRun/data/lfw.zip’

lfw.zip             100%[===================>] 187.62M  18.3MB/s    in 12s     

2021-01-02 02:16:49 (16.0 MB/s) - ‘/content/drive/MyDrive/8_ModelRun/data/lfw.zip’ saved [196739509/196739509]



In [5]:
!unzip "/content/drive/MyDrive/8_ModelRun/data/lfw.zip" -d "/content/drive/MyDrive/8_ModelRun/data/lfw"

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/8_ModelRun/data/lfw/__MACOSX/lfw/Steffi_Graf/._Steffi_Graf_0002.jpg  
  inflating: /content/drive/MyDrive/8_ModelRun/data/lfw/lfw/Steffi_Graf/Steffi_Graf_0003.jpg  
  inflating: /content/drive/MyDrive/8_ModelRun/data/lfw/__MACOSX/lfw/Steffi_Graf/._Steffi_Graf_0003.jpg  
  inflating: /content/drive/MyDrive/8_ModelRun/data/lfw/lfw/Steffi_Graf/Steffi_Graf_0004.jpg  
  inflating: /content/drive/MyDrive/8_ModelRun/data/lfw/__MACOSX/lfw/Steffi_Graf/._Steffi_Graf_0004.jpg  
  inflating: /content/drive/MyDrive/8_ModelRun/data/lfw/lfw/Steffi_Graf/Steffi_Graf_0005.jpg  
  inflating: /content/drive/MyDrive/8_ModelRun/data/lfw/__MACOSX/lfw/Steffi_Graf/._Steffi_Graf_0005.jpg  
  inflating: /content/drive/MyDrive/8_ModelRun/data/lfw/__MACOSX/lfw/._Steffi_Graf  
   creating: /content/drive/MyDrive/8_ModelRun/data/lfw/lfw/Stella_Keitel/
  inflating: /content/drive/MyDrive/8_ModelRun/data/lfw/lfw/Stella_Keitel/Stella

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
from torchvision import datasets

from PIL import Image
import torchvision.transforms as transforms
from PIL import ImageFile

import torch

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

In [15]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

### 2. Create Data Loaders for the Dog Dataset

In [8]:
transform_pipeline = transforms.Compose([transforms.RandomResizedCrop(224),
                                         transforms.ToTensor()])

def get_loader(dataset, pipeline=transform_pipeline,
               batch_size=10, num_workers=0):
    """
    This function returns a dataloader for CNN.
    It will also applied a transform pipeline, in this case:
    (1) I scaled and cropped the images to 224 using RandomResizedCrop.
    (2) I then converted the images to tensor, so they can be fed into the neural network.
    """
    df = datasets.ImageFolder('../data/dogImages/{}'.format(dataset), transform=pipeline)
    loader = torch.utils.data.DataLoader(df,
                                         batch_size=batch_size, num_workers=num_workers,
                                         shuffle=True)
    return loader

train_loader = get_loader('train')
validation_loader = get_loader('valid')
test_loader = get_loader('test')

loaders_scratch = {
    'train': train_loader,
    'valid': validation_loader,
    'test': test_loader
}

In [9]:
loaders_scratch

{'test': <torch.utils.data.dataloader.DataLoader at 0x7f849000d4a8>,
 'train': <torch.utils.data.dataloader.DataLoader at 0x7f8490038e10>,
 'valid': <torch.utils.data.dataloader.DataLoader at 0x7f849000d3c8>}

### 3. Design Model Architecture

In [3]:
class Net(nn.Module):
    
    """
    The chosen architecture is:
    (1) I defined three convolutional layers (conv1, conv2 and conv3). The first convolutional layer 
    increases depth from 3 (and RGB image depth) to 32. Each of conv2 and conv3 doubles the depth of the output 
    until it gets to 128 (so in total, depth goes from 3, to 32, 64 and 128). Each of them has convolutional 
    kernel of 3x3 and padding of 1.
    (2) I defined the max pooling layer which would downsize the XY size by 2, to discover 
    more complicated patterns in the dogs' images.
    (3) I defined a dropout layer of 0.2 to prevent overfitting.
    (4) Finally, I have two fully connected layers to output the final prediction 
    (producing 133 class scores as output).
    (5) In the forward pass function, I applied a pooling layer after applying relu to each convolutional layer. 
    For the first convolutional layer, I also applied batch normalization, 
    BatchNorm2d to normalize all inputs to have zero mean and unit variance, in order to boost CNN accuracy.
    (6) I also applied relu and pooling for the second and third convolutional layer. These convolutional 
    and pooling operations are to discover complex patterns in the dogs' images.
    (7) After that, I flattened the tensor, applied dropout and passed it to the first fully connected layer, 
    then applied the relu activation function.
    (8) Finally, I added another dropout layer to prevent overfitting and then passed the tensor to the last fully 
    connected layer to produce scores for the 133 classes.
    """
    
    def __init__(self):
        super(Net, self).__init__()
        
        n_dog_classes = 133
        n_linear_layer = 150
        
        ## Define layers of a CNN
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.norm2d1 = nn.BatchNorm2d(32)
        
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(28*28*128, n_linear_layer)
        self.fc2 = nn.Linear(n_linear_layer, n_dog_classes)
        
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        
        x = self.pool(F.relu(self.norm2d1(self.conv1(x))))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        
        x = x.view(-1, 28*28*128)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [11]:
# instantiate the CNN
model_scratch = Net()

use_cuda = True
# move tensors to GPU if CUDA is available
if use_cuda:
    model_scratch.cuda()

### 4. Specify Loss Function and Optimizer

In [12]:
### Loss Function
criterion_scratch = nn.CrossEntropyLoss()

### Select Optimizer
optimizer_scratch = optim.SGD(model_scratch.parameters(), lr=0.01)

### 5. Train and Validate the Model

In [13]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """
    Returns trained model
    """
    
    # Initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # Initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            
            # Move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            
            optimizer.zero_grad()
            
            # Forward pass:
            output = model(data)
            
            # Calculate Loss
            loss = criterion(output, target)
            
            # Back propagation
            loss.backward()
            
            # Perform optimization step
            optimizer.step()
            
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            
            # Move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            
            ## Update validation loss:
            output = model(data)
            loss = criterion(output, target)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
            
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss < valid_loss_min:
            
            torch.save(model.state_dict(), save_path)
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min, 
                                                                                            valid_loss))
            valid_loss_min = valid_loss
            
    return model

In [16]:
# Train the model
n_epochs = 50
model_scratch = train(n_epochs, loaders_scratch, model_scratch, optimizer_scratch, 
                      criterion_scratch, use_cuda, '../model/model_scratch.pt')

# Load the model that got the best validation accuracy
model_scratch.load_state_dict(torch.load('../model/model_scratch.pt'))

Epoch: 1 	Training Loss: 4.776831 	Validation Loss: 4.645805
Validation loss decreased (inf --> 4.645805).  Saving model ...
Epoch: 2 	Training Loss: 4.648361 	Validation Loss: 4.582376
Validation loss decreased (4.645805 --> 4.582376).  Saving model ...
Epoch: 3 	Training Loss: 4.573476 	Validation Loss: 4.537134
Validation loss decreased (4.582376 --> 4.537134).  Saving model ...
Epoch: 4 	Training Loss: 4.518887 	Validation Loss: 4.498446
Validation loss decreased (4.537134 --> 4.498446).  Saving model ...
Epoch: 5 	Training Loss: 4.474272 	Validation Loss: 4.465486
Validation loss decreased (4.498446 --> 4.465486).  Saving model ...
Epoch: 6 	Training Loss: 4.427686 	Validation Loss: 4.389038
Validation loss decreased (4.465486 --> 4.389038).  Saving model ...
Epoch: 7 	Training Loss: 4.389690 	Validation Loss: 4.371277
Validation loss decreased (4.389038 --> 4.371277).  Saving model ...
Epoch: 8 	Training Loss: 4.349752 	Validation Loss: 4.371613
Epoch: 9 	Training Loss: 4.296398 

<All keys matched successfully>

### 6. Test the Model

In [17]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

In [18]:
# call test function    
test(loaders_scratch, model_scratch, criterion_scratch, use_cuda)

Test Loss: 3.612915


Test Accuracy: 17% (143/836)
